In [1]:
import numpy as np
import os.path
import pickle
import pandas as pd
import torch
import matplotlib.pyplot as plt
import requests
import networkx as nx
from geopy import distance
from geopy.distance import lonlat
import random 
from collections import Counter
import dateutil
from dateutil.relativedelta import relativedelta
from requests import HTTPError
from obspy import UTCDateTime
from urllib3.exceptions import ProtocolError
import datetime
import time
from sklearn.metrics import pairwise_distances
from obspy import read as obsread
import pathlib

import os
os.environ["PROJ_LIB"] = "C:\\Users\\gabri\\AppData\\Local\\Continuum\\anaconda3\\pkgs\\proj4-5.2.0-ha925a31_1\\Library\\share";
from mpl_toolkits.basemap import Basemap

import sys
sys.path.append("../../..")
from definitions import QUAKES_RAW_DATA_PATH
from earthquakes.quakes_util import custom_geo_distance, is_point_in_bbox, get_earthquakes_in_timespan, find_closest_station, UTCDateTime_to_string, \
    string_to_UTCDateTime, quake_filename_to_UTCDateTime, BadDataException, read_pickle, write_pickle, obtain_data, timeit_context, visualize_sensors_graph, create_weighted_adjacency_quakes, \
    visualize_quakes
from utils.misc_utils import check_create_folder



In [2]:
# taken from https://www.openstreetmap.org/export
bounding_box = [
    166.104, # left lower corner longitude
    178.990, # upper right corner longitude
    -47.749, # left lower corner latitude
    -33.779, # upper right corner latitude
]

nz_basemap = Basemap(
        projection='merc',
        llcrnrlon=bounding_box[0],  # left lower corner longitude
        llcrnrlat=bounding_box[2],  # left lower corner latitude
        urcrnrlon=bounding_box[1],
        urcrnrlat=bounding_box[3],
        lat_ts=0,
        resolution='c',
        suppress_ticks=True)


C:\Users\gabri\AppData\Local\Continuum\anaconda3\envs\tud_ml4se\lib\site-packages\ipykernel_launcher.py:17: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.


In [3]:
NEW_DATASET_FOLDER = os.path.join(QUAKES_RAW_DATA_PATH, "new_dataset_gabriele")
quakes_failures_folder = os.path.join(NEW_DATASET_FOLDER, f"w=30", "quakes_failures")

In [4]:
stations_df = pd.read_csv(os.path.join(NEW_DATASET_FOLDER, "stations_in_bbox.csv"), header=None)

stations_df.head()

0   1                      2         3        4                5
0   BFZ  10  Broadband Seismometer  176.2462 -40.6796   7/30/2003 1:00
1   BHW  10  Broadband Seismometer  174.8711 -41.4082  7/14/2016 22:30
2   BKZ  10  Broadband Seismometer  176.4925 -39.1657  2/10/2004 21:00
3  COVZ  10  Broadband Seismometer  175.5424 -39.1999  11/1/2009 18:31
4   CVZ  10  Broadband Seismometer  171.0061 -44.3831  5/15/2014 20:00

In [52]:
failures = [(file, read_pickle(os.path.join(quakes_failures_folder,file))) for file in os.listdir(quakes_failures_folder) if file.endswith(".pickle")]
print(f"{len(failures)} failures found.")

failures_dict = {}
failures_to_delete = []
for filename, description in failures:
    if "No Content for url" in description:
        failing_station = description.split("station=")[1].split("&")[0]
        failing_year_month = description.split("starttime=")[1][:7]
        failures_dict[failing_station] = failures_dict.get(failing_station, 0) + 1
    elif "Max retries exceeded" in description:
        failures_to_delete.append((filename, description))
    elif "Unknown format for file" in description:
        pass
    elif "BadDataException" in description:
        pass 
    elif "OSError" in description:
        failures_to_delete.append((filename, description)) 
    else:
        raise Exception(description)

print(len(failures_to_delete))

for filename, description in failures_to_delete:
    filepath_to_remove = os.path.join(quakes_failures_folder, filename)
    os.remove(filepath_to_remove)

10665 failures found.
0


--------------

--------------

--------------

In [6]:
stations_to_query = list(stations_df[0].values)

In [21]:
API_TO_TEST = "https://service.geonet.org.nz/fdsnws/dataselect/1/query?network=NZ&station={}&location=10&channel=HHZ&starttime=2013-04-08T06:43:15&endtime=2013-04-08T06:44:15"

mseeds = []
failed_stations = []
for selected_station in stations_to_query:
    query_url = API_TO_TEST.format(selected_station)
    
    try:
        mseed_for_station = obsread(query_url)[0]
        mseeds.append(mseed_for_station)
    except (HTTPError, TypeError) as e:
        print(e)
        mseeds.append(None)
        failed_stations.append(selected_station)
    

print(failed_stations)

204 HTTP Error: No Content for url: https://service.geonet.org.nz/fdsnws/dataselect/1/query?network=NZ&station=BHW&location=10&channel=HHZ&starttime=2013-04-08T06:43:15&endtime=2013-04-08T06:44:15
204 HTTP Error: No Content for url: https://service.geonet.org.nz/fdsnws/dataselect/1/query?network=NZ&station=CVZ&location=10&channel=HHZ&starttime=2013-04-08T06:43:15&endtime=2013-04-08T06:44:15
204 HTTP Error: No Content for url: https://service.geonet.org.nz/fdsnws/dataselect/1/query?network=NZ&station=ETVZ&location=10&channel=HHZ&starttime=2013-04-08T06:43:15&endtime=2013-04-08T06:44:15
Unknown format for file C:\Users\gabri\AppData\Local\Temp\obspy-j681v_ou.tmp
204 HTTP Error: No Content for url: https://service.geonet.org.nz/fdsnws/dataselect/1/query?network=NZ&station=TMVZ&location=10&channel=HHZ&starttime=2013-04-08T06:43:15&endtime=2013-04-08T06:44:15
Unknown format for file C:\Users\gabri\AppData\Local\Temp\obspy-co1pm4v8.tmp
['BHW', 'CVZ', 'ETVZ', 'FWVZ', 'TMVZ', 'WSRZ']


In [ ]:
failed_stations_df= stations_df[stations_df[0].isin(failed_stations)]

failed_station_lats=list(failed_stations_df[4].values)
failed_station_lons=list(failed_stations_df[3].values)

plt.figure(figsize=(6, 6))



mapped_x, mapped_y = nz_basemap(failed_station_lons, failed_station_lats)
g = nx.Graph()

positions={}
for idx in range(len(mapped_x)):
    g.add_edge(idx, idx)
    positions[idx] = (mapped_x[idx], mapped_y[idx])

nx.draw_networkx(g, positions, node_size=50,node_color="white", with_labels=False, alpha=1)
nz_basemap.bluemarble()
plt.show()